# Regression: Canadian Oil and Gas Stock Prediction using Company Balance Sheets

### Introduction

In this project we want to create a stock price prediction model using company balance sheet values for various Canadian oil and gas companies. We will be using the Python Yahoo Finance API to get the data, then we would be utilizing Python for data wrangling and cleaning, then we would switch into R, where we would go ahead and attempt to create a powerful regression model to predict the stock price off of a given oil and gas company's balance sheet values. From my accounting knowledge, it was decided that utilizing a balance sheet for stock price prediction would make the most sense as this is the one company financial statement which provides a snapshot in time, whereas other financial statements provide values over a certain period of time. This regression model would then be able to be used for a strategy of value investing within the oil and gas industry.

Value Investing: An investment strategy based on looking for stocks which are undervalued by the market at large.

### Data Cleaning and Wrangling

In [1]:
# importing relevant packages
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
# suppresses scientific notation for pandas dataframes
pd.options.display.float_format = '{:.0f}'.format

In [2]:
def date_correction(date, stock_prices):
    """
    this function takes a date and then checks if it is present in another pandas series, if it is not,
    we subtract one day and then check again, over and over, until the date is present in the second
    series, and then returns this new date (important as balance sheets are not always recorded on stock
    trading days, therefore the best estimate would be the last stock value right before the balance sheet date)
    """
    while date not in stock_prices.values:
        date -= pd.DateOffset(days = 1)

    return date

# creating a list of relevant canadian oil and gas stock tickers
tickers = ["SU", "CVE", "CNQ", "TRP", "IMO", "ENB", "PPL", "KEY", "XOM", "CVX", "SHEL", "TTE", "COP", "BP",
           "MPC", "OXY", "PXD", "KMI", "WDS", "WMB", "LNG", "MPLX", "SLB", "EOG", "TOU", "SUN", "VLO", "PKN", "OVV"]

# initializing an empty list to store the final merged dataframes (each element is for 1 stock ticker) to combine
merged_dfs = []

# looping through each stock ticker in the list above
for ticker in tickers:
    
    # initializing the variable for yahoo finance information for the current stock ticker
    temp_ticker_info = yf.Ticker(ticker)

    # extracting the balance sheet information from the yahoo finance api and transposing it to format it correctly
    balance_sheet = temp_ticker_info.quarterly_balance_sheet.transpose()
    # removing the date index and renaming it to "Date" as a regular column
    balance_sheet = balance_sheet.reset_index().rename(columns = {"index":"Date"})
    
    # extracting the stock price information from the yahoo finance api starting from the date "2015-01-01"
    # as we only want recent information
    stock_price = temp_ticker_info.history(start = "2015-01-01", end = None)
    # removing the date index and renaming it to "Date" as a regular column
    stock_price = stock_price.reset_index().rename(columns = {"index":"Date"})
    # removing the time from the date column
    stock_price["Date"] = pd.to_datetime(stock_price["Date"]).dt.date
    # filtering for only the "Date" and "Close" (closing stock price) columns
    stock_price = stock_price[["Date", "Close"]]
    # renaming the closing stock price column
    stock_price = stock_price.rename(columns = {"Close":"ClosingStockPrice"})
    # converting the "Date" column to a datetime64[ns] object
    stock_price["Date"] = stock_price["Date"].astype("datetime64[ns]")

    # subtracting days from balance sheet dates, until we get to a date on which the stock market was open, to match up
    # stock prices with balance sheet numbers for balance sheets issued out on non-trading days
    balance_sheet["Date"] = balance_sheet["Date"].apply(lambda x: date_correction(x, stock_price["Date"]))

    # merging the balance_sheet dataframe and the stock_price dataframe on the "Date" column
    # left join as we only want matching stock price values, but all balance sheet values
    merged = pd.merge(balance_sheet, stock_price, on = "Date", how = "left")

    # appending the merged dataframe to the merged_dfs list
    merged_dfs.append(merged)
    
# concatenating all of the dataframes for each stock ticker into one
stock_price_prediction = pd.concat(merged_dfs)

# resetting the index values for the dataframe
stock_price_prediction = stock_price_prediction.reset_index()
# dropping the new index column
stock_price_prediction = stock_price_prediction.drop(columns = ["index"])

In [3]:
# deleting the rows where the closing value for the stock price is missing, as it is not useful for our regression model
# (due to this being our predictor variable)
stock_price_prediction.dropna(subset = ["ClosingStockPrice"], inplace = True)

# setting a threshold value of having over 15% of missing values in a column
threshold = stock_price_prediction.shape[0]*0.15
# creating a series object of how many missing values each column has
is_na = stock_price_prediction.isna().sum()
# storing the columns that are above the threshold of 15% missing values
over_15pct_columns = is_na[is_na > threshold]
# convering the columns with over 15% missing values to a list format
missing_columns = over_15pct_columns.index.to_list()
# dropping these columns as they are not valuable
stock_price_prediction = stock_price_prediction.drop(columns = missing_columns)

In [4]:
# getting a series object of a count of missing values in each row
is_na = stock_price_prediction.isna().sum(axis = 1)
# filtering for index values for all rows with any missing values
missing_index = is_na[is_na > 0].index.to_list()
# dropping rows with any missing values, as we need all variables in order to 
# train and run our regression model
stock_price_prediction = stock_price_prediction.drop(missing_index)

In [5]:
# removing all non-numeric columns and adding these to a new list variable
columns = stock_price_prediction.columns.to_list()
columns.remove("Date")
# looping through all numeric columns in the dataframe and making them into a numeric datatype
for column in columns:
    stock_price_prediction[str(column)] = pd.to_numeric(stock_price_prediction[str(column)])

In [6]:
# displaying the dataframe in order to write information about the columns
stock_price_prediction.head()

,Date,Ordinary Shares Number,Share Issued,Net Debt,Total Debt,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,Common Stock Equity,...,Total Non Current Assets,Other Non Current Assets,Net PPE,Current Assets,Inventory,Receivables,Accounts Receivable,Cash Cash Equivalents And Short Term Investments,Cash And Cash Equivalents,ClosingStockPrice
0,2023-09-29,1297798000,1297798000,9837000000,15427000000,38229000000,54039000000,2993000000,38229000000,41770000000,...,70078000000,2023000000,64396000000,15311000000,5186000000,7693000000,7315000000,2432000000,2432000000,34
1,2023-06-30,1302117000,1302117000,11170000000,17004000000,37262000000,54599000000,2133000000,37262000000,40819000000,...,70846000000,1712000000,65482000000,14135000000,5008000000,6517000000,6007000000,2610000000,2610000000,29
2,2023-03-31,1321913697,1321913697,12439000000,16842000000,36384000000,53516000000,1348000000,36384000000,39949000000,...,70871000000,1779000000,65443000000,13885000000,5230000000,6728000000,6435000000,1128000000,1128000000,30
3,2022-12-30,1337470739,1337470739,10627000000,15619000000,35781000000,51974000000,1667000000,35781000000,39367000000,...,70082000000,1766000000,64649000000,14536000000,5058000000,6312000000,6068000000,1980000000,1980000000,30
4,2023-09-29,1885559000,1885559000,5976000000,9971000000,25361000000,35533000000,3540000000,25880000000,28295000000,...,43682000000,288000000,39261000000,10745000000,4816000000,4667000000,4534000000,1262000000,1262000000,21


### Dataset:

Date

Ordinary Shares Number

Share Issued

Net Debt

Total Debt

Tangible Book Value

Invested Capital

Working Capital

Net Tangible Assets

Common Stock Equity

Total Capitalization

Total Equity Gross Minority Interest

Stockholders Equity

Capital Stock

Common Stock

Total Liabilities Net Minority Interest

Total Non Current Liabilities Net Minority Interest

Other Non Current Liabilities

Non Current Deferred Liabilities

Non Current Deferred Taxes Liabilities

Long Term Debt And Capital Lease Obligation

Long Term Debt

Current Liabilities

Current Debt And Capital Lease Obligation

Payables And Accrued Expenses

Payables

Accounts Payable

Total Assets

Total Non Current Assets

Other Non Current Assets

Net PPE

Current Assets

Inventory

Receivables

Accounts Receivable

Cash Cash Equivalents And Short Term Investments

Cash And Cash Equivalents

Stock Price

Stock Ticker

In [7]:
# exporting the final financial statements dataframe into a csv format for statistical modelling in R
stock_price_prediction.to_csv("stock_price_prediction_oil_gas.csv")

Now, we can move into R and conduct our regression modelling.

### References

Yfinance. PyPI. (2023). https://pypi.org/project/yfinance/